### 1. https://docs.scala-lang.org/overviews/quasiquotes/setup.html    
Scala 2.11, quasiquotes 는 scala-reflect.jar의 부분으로 스파크 공식 배포로 이동되었다.    
그래서 특별히 사용하기 위해서 뭔가 할 필요 없다. 그저 **scala-reflect**만 추가하면 된단다.    


In [44]:
val universe: scala.reflect.runtime.universe.type = scala.reflect.runtime.universe

universe = scala.reflect.runtime.JavaUniverse@3d472b86


scala.reflect.runtime.JavaUniverse@3d472b86

In [45]:
import universe._

universe의 와일드 카드 임포트가 우리가 quasiquotes를 사용하기 위해 할 일이다.     
추가적으로 사용할 **ToolBox** API 정도

In [46]:
import scala.reflect.runtime.currentMirror
import scala.tools.reflect.ToolBox
val toolbox = currentMirror.mkToolBox()

toolbox = scala.tools.reflect.ToolBoxFactory$ToolBoxImpl@961d685


scala.tools.reflect.ToolBoxFactory$ToolBoxImpl@961d685

새롭고 반짝반짝한 **showCode** 프리티 프린터

In [47]:
val C = q"class C"

C = 


class C extends scala.AnyRef {
  def <init>() = {
    super.<init>();
    ()
  }
}


In [48]:
println(showCode(C))

class C


### 2. https://docs.scala-lang.org/overviews/quasiquotes/intro.html
quasiquotes은 Scala Syntax Tree를 좀 더 쉽게 조작 할 수 있도록 하는 깔끔한 표기법이다.    


In [49]:
val tree = q"i am { a quasiquote }"

tree = i.am(a.quasiquote)


i.am(a.quasiquote)

코드 조각을 q를 사용해서 랩핑하면 주어진 코드 조각을 표현하는 트리가 된다.    
니가 이미 알다시피, quotation 구문은 그저 문자열 보간법의 다른 사용 형태일 뿐이다. 2.10 버전에 도입된   
그저 문자열로 보이지만 후드 아래에서 구문 트리 아래에서 실행된다.

In [50]:
println(tree match { case q"i am { a quasiquote }" => "it worked!" })

it worked!


트리와 quasiquote를 매치 할 때마다 이것은 주어진 트리의 구조가 주어진 패턴과 일치하는지 매치한다.    
equalsStructure 메소드의 도움으로 매뉴얼하게 구조적으로 일치하는지 체크할 수 있다.

In [51]:
println(q"foo + bar" equalsStructure q"foo.+(bar)")

true


$를 사용해서 quasiquotation에 넣을 수도 있다.

In [52]:
val aquasiquote = q"a quasiquote"

aquasiquote = a.quasiquote


a.quasiquote

In [53]:
val tree = q"i am { $aquasiquote }"

tree = i.am(a.quasiquote)


i.am(a.quasiquote)

이 연산자는 unquoting으로도 알려져 있다.    
quasiquote 내에서 타입 트리 표현식을 unquote 할 때 마다 구조적으로 트리를 그 위치에서 대체한다.    
대부분 quote들 간 대체에서 소스 코드의 텍스트 대체와 동등하다.    
비슷하게도, 패턴 매칭 내 unquoting으로 구조적으로 트리 해체를 할 수 있다. 

In [54]:
val q"i am $what" = q"i am { a quasiquote }"

what = a.quasiquote


a.quasiquote

**Interpolators**    
스칼라는 풍부한 구문을 가진 언어로, 통사적 맥락에 따라 크게 달라진다.

In [55]:
val x = q"""
        val x: List[Int] = List(1, 2) match {
            case List(a, b) => List(a + b)
        }
"""

x = 


val x: List[Int] = List(1, 2) match {
  case List((a @ _), (b @ _)) => List(a.$plus(b))
}


||||
|---|---|---|
||Used for| |
|q | expression, definitions and imports ||
|tq | types ||
|pq | patterns ||
|cq | case clause | |
|fq | for loop enumerator | |
서로 다른 컨텍스트들 간의 구문적인 유사도가 아래의 두 트리의 유사도를 함축하지는 않는다.

In [56]:
println(q"List[Int]" equalsStructure tq"List[Int]")

false


In [57]:
println(showRaw(q"List[Int]"))

TypeApply(Ident(TermName("List")), List(Ident(TypeName("Int"))))


In [58]:
println(showRaw(tq"List[Int]"))

AppliedTypeTree(Ident(TypeName("List")), List(Ident(TypeName("Int"))))


In [59]:
println(showRaw(pq"List[int]"))

AppliedTypeTree(Ident(TypeName("List")), List(Bind(TypeName("int"), EmptyTree)))


In [60]:
println(pq"List(a, b)" equalsStructure q"List(a, b)")

false


 **Splicing**     
 Unquote Splicing은 다양한 수의 엘리먼트 변수를 unquote하는 방법이다.

In [61]:
val ab = List(q"a", q"b")
val fab = q"f(..$ab)"

ab = List(a, b)
fab = f(a, b)


f(a, b)

unquotee 명세 이전의 도트는 flattening의 정도를 나타낸다. 그리고 splicing rank 라고 불린다 ..$     
..$는 argument가 Iterable[Tree] 로 예상하고 ...$는 Iterable[Iterable[Tree]] 로 예상한다.   
splicing은 regular unquotation과 간편하게 결합될 수 있다.

In [62]:
val c = q"c"
val fabc = q"f(..$ab, $c)"

c = c
fabc = f(a, b, c)


f(a, b, c)

In [63]:
val fcab = q"f($c, ..$ab)"

fcab = f(c, a, b)


f(c, a, b)

In [64]:
val fabcab = q"f(..$ab, $c, ..$ab)"

fabcab = f(a, b, c, a, b)


f(a, b, c, a, b)

만약 Application 에 대해 추상화하고 싶으면 ...$ 를 사용할 수 있다

In [65]:
val argss = List(ab, List(c))

argss = List(List(a, b), List(c))


List(List(a, b), List(c))

In [66]:
val fagss = q"f(...$argss)"

fagss = f(a, b)(c)


f(a, b)(c)

...$ 의 splicing 은 함수 application과 def 와 class 정의의 파라미터 리스트에만 지원된다.

In [67]:
val q"f(..$args)" = q"f(a, b)"

args = List(a, b)


List(a, b)

In [68]:
val q"f(...$argss)" = q"f(a, b)(c)"

argss = List(List(a, b), List(c))


List(List(a, b), List(c))

splicing과 $ 변수 실행을 제한하는 몇가지 방법이 있다.

In [69]:
case q"f($first, ..$rest)"  => // ok
case q"f(..$init, $last)"   => // ok
case q"f(..$a, ..$b)"       => // not allowed

Name: Compile Error
Message: <console>:1: error: illegal start of definition
case q"f($first, ..$rest)"  => // ok
^

StackTrace: 

주어진 리스트에서 오직 한 ..$ 만 허용된다. ...$도 마찬가지다

3. https://docs.scala-lang.org/overviews/quasiquotes/lifting.html
lifting은 quasiquotes 내부의 커스텀 데이터 타입을 unquote(디코딩)하는 확장 가능한 방법이다.    
이것의 주요한 use-case는 literal 값과 트리의 여러 요소들 unquoting의 지원이다.

In [70]:
val two = 1 + 1
val four = q"$two + $two"

two = 2
four = 2.$plus(2)


2.$plus(2)

위 코드는 성공적으로 동작한다. 그 이유는 Int는 기본적으로 Liftable 하기 때문이다.    
Liftable 타입은 trait이다. 트리에 주어진 타입에 매핑된 단일 추상화 메소드다

In [71]:
import scala.reflect.runtime.universe._

trait Liftable[T] {
    def apply(value: T): Tree
}

defined trait Liftable


In [72]:
val ints = List(1, 2, 3)
val f123 = q"f(..$ints)"

ints = List(1, 2, 3)
f123 = f(1, 2, 3)


f(1, 2, 3)

In [73]:
val intss = List(List(1, 2, 3), List(4, 5), List(6))
val f123456 = q"f(...$intss)"

intss = List(List(1, 2, 3), List(4, 5), List(6))
f123456 = f(1, 2, 3)(4, 5)(6)


f(1, 2, 3)(4, 5)(6)

In [74]:
case class Point(x : Int, y: Int)
object Point {
    implicit val lift = Liftable[Point] { 
        p => q"_root_.points.Point(${p.x}, ${p.y})"
    }
}

Name: Compile Error
Message: <console>:15: error: not found: value Liftable
           implicit val lift = Liftable[Point] {
                               ^
<console>:16: error: value q is not a member of StringContext
 Note: implicit value lift is not applicable here because it comes after the application point and it lacks an explicit result type
               p => q"_root_.points.Point(${p.x}, ${p.y})"
                    ^

StackTrace: 